In [1]:
import numpy as np
import pandas as pd

import os
import glob
from datetime import datetime

In [2]:
# 1개 sao 파일을 열어서 내용 읽고 데이터프레임으로 변환하는 함수
# 참조 - https://stackabuse.com/read-a-file-line-by-line-in-python/
def getData(FILENAME):
    with open(FILENAME) as fp:
        # 파일 내용 한줄씩 읽기
        line = fp.readline()
        
        cnt = 0
        check = []
        lines = []
        
        while line:           
            # line 내용이 공백없이 연속되고, 시작글자가 FF인 라인을 찾아 check에 인덱스 저장
            if (len(line.strip().split()) == 1) & (line.strip().split()[0][:2] == 'FF'):
                check.append(cnt)
            
            # 파일 내용 lines에 한줄씩 저장
            lines.append(line)
            
            line = fp.readline()
            cnt += 1
    
    # 데이터를 담을 딕셔너리
    data_dict = {
        'Time' : [],
        'Receiver' : [],
        'Transmitter' : [],
        'foF2' : [],
        'hmF2' : [],
        'Confidence level' : []
    }

    # check에 저장된 인덱스로 lines에서 데이터 추출
    for idx in check:

        # 1. 시간 데이터
        a1 = int(lines[idx][2:6].strip())
        a2 = int(lines[idx][9:11].strip())
        a3 = int(lines[idx][11:13].strip())
        a4 = int(lines[idx][13:15].strip())
        a5 = int(lines[idx][15:17].strip())
        a6 = int(lines[idx][17:19].strip())
        data1 = datetime(a1, a2, a3, a4, a5, a6)        
        
        # 2. 송신기
        data2 = int(lines[idx][19:22].strip())
        
        # 3. 수신기
        data3 = int(lines[idx][22:25].strip())
        
        # 4. foF2
        data4 = float(lines[idx+1][:8].strip())
        
        # 5. hmF2
        data5 = float(lines[idx+3][8:16].strip())
        
        # 6. confidence level
        data6 = int(lines[idx+5][18:20].strip())

        # 딕셔너리에 추출한 여섯가지 데이터 저장
        data_dict['Time'].append(data1)
        data_dict['Receiver'].append(data2)
        data_dict['Transmitter'].append(data3)
        data_dict['foF2'].append(data4)
        data_dict['hmF2'].append(data5)
        data_dict['Confidence level'].append(data6)

    # 딕셔너리를 데이터프레임(시계열)으로 변환하고 시간순으로 정렬
    df = pd.DataFrame(data_dict)
    df = df.sort_values('Time')
        
    return df

In [3]:
# 폴더명을 받아 하위 sao 파일을 차례로 읽으면서 데이터프레임으로 합치는 함수
def getCombinedData(FORDER_NAME):
    
    # 입력 받은 폴더의 하위 파일명 리스트
    FILE_LIST = os.listdir(FORDER_NAME)
    
    # 데이터프레임 생성
    df = pd.DataFrame()
    # 파일명 리스트로부터 파일명을 하나씩 읽어 getData함수에 입력
    for FILE_NAME in FILE_LIST:
        
        FILE_NAME = FORDER_NAME + "/" + FILE_NAME
    
        # 입력된 파일명으로부터 데이터가 담긴 데이터프레임(시계열) 반환
        df_new = getData(FILE_NAME)
        
        # 기존 데이터프레임에 새로 반환된 데이터프레임 합치기
        df = pd.concat((df, df_new), axis=0)
        
    # 합쳐진 데이터프레임 csv 파일로 저장    
    SAVE_PATH = 'data/'
    SAVE_NAME = FORDER_NAME.split('/')[-1].replace('SAO', '')
    df.to_csv(f'{SAVE_PATH}{SAVE_NAME}.csv', encoding='utf-8-sig', index=False)

    return df

In [4]:
STATION_CODE = 'JJ433'
PATH = f"data/ionosonde/{STATION_CODE}*"

FORDER_LIST = glob.glob(PATH)
for FORDER_NAME in FORDER_LIST:
    FORDER_NAME = FORDER_NAME.replace('\\','/')
    getCombinedData(FORDER_NAME)
    print(f'{FORDER_NAME} 완료')
    break

data/ionosonde/JJ433SAO_2009 완료
